In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import os
import pandas as pd 
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import  hstack
from astropy.table import  vstack
from astropy.table import unique
from astropy.table import Column
from astropy.table import join
import celerite
import imp
mod = imp.load_source('modules.py', '../code/modules.py')
from matplotlib import rcParams 
rcParams['ytick.labelsize'] = 15
rcParams['xtick.labelsize'] = 15
rcParams['axes.labelsize'] = 20
rcParams['axes.linewidth'] = 2
rcParams['font.size'] = 15
rcParams['axes.titlesize'] = 18

In [83]:
outDir = '../dp2/Fig2_simLC_SDSS-PS1/'
simLcs =  os.listdir(outDir)


In [36]:


def calc_Fig2_data(lcname):
    outDir = '../dp2/Fig2_simLC_SDSS-PS1/'
    simulated = np.load(outDir + lcname).item()

    # do just the SDSS-PS1 part first 
    sig_lims , tau_lims = [0.02,0.7], [1,5000]


    tau_min = simulated[0]['tau_in']
    tau_max = simulated[99]['tau_in']

    tau_lims = [tau_min-1, tau_max+100]
    sig_lims = [0.02, 0.7] # since sigma_in  is 0.14 for all LCs...
    sig_in = 0.14
    tau_in = 100 # starting values ... 
    prior = 'Jeff1'  # 1/sigma * 1/tau  
    t = simulated['mjd']
    yerr = simulated['y_err']

    irange = 100
    jrange = 100 

    results = { 'sigmaMAP' : np.zeros(irange*jrange, dtype=float),
               'sigmaEXP' : np.zeros(irange*jrange, dtype=float),
               'tauMAP' : np.zeros(irange*jrange, dtype=float),
               'tauEXP' : np.zeros(irange*jrange, dtype=float),
               'rhoIN' : np.zeros(irange*jrange, dtype=float),
               'tauIN' : np.zeros(irange*jrange, dtype=float),
              }

    k = 0 
    for i in range(irange):  # rho grid 
        for j in range(jrange) :  # j-th realization .. 
            if j % 10 == 0 :  print(lcname, i , j )

            y = simulated[i][j] # the observed magnitude 
            # find the Celerite expectation value and MAP 
            # find the MAP estimate 
            sigmaMAP, tauMAP,gp = mod.find_celerite_MAP(t ,y,yerr, 
                                      sig_in, tau_in, prior=prior,set_bounds=True, 
                                      sig_lims=sig_lims, tau_lims=tau_lims ,
                                                        verbose=False)
            # expectation from grid  
            Ngrid = 60  ; scale = 'log'
            sigma_grid, tau_grid = mod.make_grid(scale,sig_lims, tau_lims,Ngrid) 
            logP = mod.evaluate_logP(sigma_grid, tau_grid, y, gp, prior, 'celerite')

            # find the expectation value 
            sigmaEXP, tauEXP = mod.find_expectation_value(logP, sigma_grid,  tau_grid)

            # store the results 
            results['sigmaMAP'][k] = sigmaMAP
            results['sigmaEXP'][k] = sigmaEXP
            results['tauMAP'][k] = tauMAP
            results['tauEXP'][k] = tauEXP
            results['rhoIN'][k] = simulated[i]['rho_in']
            results['tauIN'][k] = simulated[i]['tau_in']

            # update the counter 
            k += 1 
    print('Done')

    # Store the results ...
    resDir = '../dp2/Fig2_results_SDSS-PS1/'

    np.save(resDir + lcname[:-4] , results , )
    print('Saved all fitted data for Fig2 for that set of cadence and errors for SDSS-PS1')
    print('  to  ',resDir + lcname[:-4]+'.npy') 

    return 

In [ ]:
# simulated LCs - 10000 per real SDSS-PS1 cadence / errors ... 
outDir = '../dp2/Fig2_simLC_SDSS-PS1/'
simLcs =  os.listdir(outDir)

# exclude those lcs that 
# have already been calculated 
resDir = '../dp2/Fig2_results_SDSS-PS1/'


select_not_done = ~np.in1d(simLcs,  os.listdir(resDir))
print(np.sum(select_not_done), '/', len(select_not_done), ' are not yet calculated')
# exclude "catastrophic failures"...
real = Table.read('190409_Celerite_real_Jeff1.txt', format='ascii')
lcname_npy = np.array([lc[:-4]+'.npy' for lc in real['lcname'].data.data])
lcname_good = lcname_npy[np.log10(real['tauEXP_sdss-ps1']) > 1.5 ]
select_not_catastrophic = np.in1d(simLcs, lcname_good)
print(np.sum(select_not_catastrophic), '/', len(select_not_catastrophic), 
      ' are not failures (log10(tauEXP) > 1.5 ) ')

simLcs_to_do = np.array(simLcs)[select_not_done & select_not_catastrophic]
print('In total, we are processing ', len(simLcs_to_do), ' of ',  len(simLcs))
for ith in range(len(simLcs_to_do)):
    print(ith,'/',len(simLcs_to_do))
    lcname = simLcs_to_do[ith]
    calc_Fig2_data(lcname)

96 / 100  are not yet calculated
69 / 100  are not failures (log10(tauEXP) > 1.5 ) 
In total, we are processing  66  of  100


/Users/chris/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log10
  from ipykernel import kernelapp as app


1025069.npy 0 0
1025069.npy 0 10
1025069.npy 0 20
1025069.npy 0 30
1025069.npy 0 40
1025069.npy 0 50
1025069.npy 0 60
1025069.npy 0 70
1025069.npy 0 80
1025069.npy 0 90
1025069.npy 1 0
1025069.npy 1 10
1025069.npy 1 20
1025069.npy 1 30
1025069.npy 1 40
1025069.npy 1 50
1025069.npy 1 60
1025069.npy 1 70
1025069.npy 1 80
1025069.npy 1 90
1025069.npy 2 0
1025069.npy 2 10
1025069.npy 2 20
1025069.npy 2 30
1025069.npy 2 40
1025069.npy 2 50
1025069.npy 2 60
1025069.npy 2 70
1025069.npy 2 80
1025069.npy 2 90
1025069.npy 3 0
1025069.npy 3 10
1025069.npy 3 20
1025069.npy 3 30
1025069.npy 3 40
1025069.npy 3 50
1025069.npy 3 60
1025069.npy 3 70
1025069.npy 3 80
1025069.npy 3 90
1025069.npy 4 0
1025069.npy 4 10
1025069.npy 4 20
1025069.npy 4 30
1025069.npy 4 40
1025069.npy 4 50
1025069.npy 4 60
1025069.npy 4 70
1025069.npy 4 80
1025069.npy 4 90
1025069.npy 5 0
1025069.npy 5 10
1025069.npy 5 20
1025069.npy 5 30
1025069.npy 5 40
1025069.npy 5 50
1025069.npy 5 60
1025069.npy 5 70
1025069.npy 5 80
102

1025069.npy 46 40
1025069.npy 46 50
1025069.npy 46 60
1025069.npy 46 70
1025069.npy 46 80
1025069.npy 46 90
1025069.npy 47 0
1025069.npy 47 10
1025069.npy 47 20
1025069.npy 47 30
1025069.npy 47 40
1025069.npy 47 50
1025069.npy 47 60
1025069.npy 47 70
1025069.npy 47 80
1025069.npy 47 90
1025069.npy 48 0
1025069.npy 48 10
1025069.npy 48 20
1025069.npy 48 30
1025069.npy 48 40
1025069.npy 48 50
1025069.npy 48 60
1025069.npy 48 70
1025069.npy 48 80
1025069.npy 48 90
1025069.npy 49 0
1025069.npy 49 10
1025069.npy 49 20
1025069.npy 49 30
1025069.npy 49 40
1025069.npy 49 50
1025069.npy 49 60
1025069.npy 49 70
1025069.npy 49 80
1025069.npy 49 90
1025069.npy 50 0
1025069.npy 50 10
1025069.npy 50 20
1025069.npy 50 30
1025069.npy 50 40
1025069.npy 50 50
1025069.npy 50 60
1025069.npy 50 70
1025069.npy 50 80
1025069.npy 50 90
1025069.npy 51 0
1025069.npy 51 10
1025069.npy 51 20
1025069.npy 51 30
1025069.npy 51 40
1025069.npy 51 50
1025069.npy 51 60
1025069.npy 51 70
1025069.npy 51 80
1025069.npy 51 

1025069.npy 92 20
1025069.npy 92 30
1025069.npy 92 40
1025069.npy 92 50
1025069.npy 92 60
1025069.npy 92 70
1025069.npy 92 80
1025069.npy 92 90
1025069.npy 93 0
1025069.npy 93 10
1025069.npy 93 20
1025069.npy 93 30
1025069.npy 93 40
1025069.npy 93 50
1025069.npy 93 60
1025069.npy 93 70
1025069.npy 93 80
1025069.npy 93 90
1025069.npy 94 0
1025069.npy 94 10
1025069.npy 94 20
1025069.npy 94 30
1025069.npy 94 40
1025069.npy 94 50
1025069.npy 94 60
1025069.npy 94 70
1025069.npy 94 80
1025069.npy 94 90
1025069.npy 95 0
1025069.npy 95 10
1025069.npy 95 20
1025069.npy 95 30
1025069.npy 95 40
1025069.npy 95 50
1025069.npy 95 60
1025069.npy 95 70
1025069.npy 95 80
1025069.npy 95 90
1025069.npy 96 0
1025069.npy 96 10
1025069.npy 96 20
1025069.npy 96 30
1025069.npy 96 40
1025069.npy 96 50
1025069.npy 96 60
1025069.npy 96 70
1025069.npy 96 80
1025069.npy 96 90
1025069.npy 97 0
1025069.npy 97 10
1025069.npy 97 20
1025069.npy 97 30
1025069.npy 97 40
1025069.npy 97 50
1025069.npy 97 60
1025069.npy 97 

3989384.npy 37 80
3989384.npy 37 90
3989384.npy 38 0
3989384.npy 38 10
3989384.npy 38 20
3989384.npy 38 30
3989384.npy 38 40
3989384.npy 38 50
3989384.npy 38 60
3989384.npy 38 70
3989384.npy 38 80
3989384.npy 38 90
3989384.npy 39 0
3989384.npy 39 10
3989384.npy 39 20
3989384.npy 39 30
3989384.npy 39 40
3989384.npy 39 50
3989384.npy 39 60
3989384.npy 39 70
3989384.npy 39 80
3989384.npy 39 90
3989384.npy 40 0
3989384.npy 40 10
3989384.npy 40 20
3989384.npy 40 30
3989384.npy 40 40
3989384.npy 40 50
3989384.npy 40 60
3989384.npy 40 70
3989384.npy 40 80
3989384.npy 40 90
3989384.npy 41 0
3989384.npy 41 10
3989384.npy 41 20
3989384.npy 41 30
3989384.npy 41 40
3989384.npy 41 50
3989384.npy 41 60
3989384.npy 41 70
3989384.npy 41 80
3989384.npy 41 90
3989384.npy 42 0
3989384.npy 42 10
3989384.npy 42 20
3989384.npy 42 30
3989384.npy 42 40
3989384.npy 42 50
3989384.npy 42 60
3989384.npy 42 70
3989384.npy 42 80
3989384.npy 42 90
3989384.npy 43 0
3989384.npy 43 10
3989384.npy 43 20
3989384.npy 43 3

In [82]:
lcname

'1025069.npy'